In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys
import dgl
sys.path.append('.')

from glob import glob
from configs import *
from tqdm import tqdm
from utils import *
import numpy as np
# from dataset import *k
from days import *
from time import time
tqdm.monitor_interval = 0
import torch
import json
from time import time
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

In [3]:
groups = [
    
#     '/beegfs/ua349/logs/slurm_1353164.out',
    ['Linear', '/beegfs/ua349/logs/linear'],
    ['GCN', '/beegfs/ua349/logs/gcn'],
]

In [10]:
hops = [
    2,
    3,
    4,
    5
]

In [11]:
delim = ' '
for name, folder in groups:
    hop_avg = []
    for hop in range(1, 6):
        if hop in hops:
            avg = []
            for logfl in glob('%s/*n%d*' % (folder, hop)):
                with open(logfl) as fl:
                    res = json.load(fl)
                hist, best_eval = res
                avg.append(np.sqrt(best_eval))
            avg = np.mean(avg)
            hop_avg.append(avg)
        else:
            hop_avg.append(None)
    out = delim.join([
        name,
        delim.join(['%.4f' % avg if avg is not None else '-' for avg in hop_avg]),
    ])
    print(out)

Linear - 0.2934 0.3003 0.3089 0.3205
GCN - 0.2941 0.2936 0.2962 0.2979


In [14]:
!python3 jobs/train_rnn.py "/beegfs/ua349/archive/data/graphs/400861-400948_n2.json"

Saving to:
/beegfs/ua349/checkpoints/mxrnn/400861-400948_n2.pth
Full history
 [*] Segments: 90 co-avail
    * [400861-400948]: 90 (25.5% nan)
    * [400860-400861]: 90 (6.1% nan)
    * [400948-404050]: 90 (34.5% nan)
    * [404050-400950]: 90 (41.9% nan)
    * [401348-400860]: 90 (45.4% nan)
 [*] Examples (train): 72
 [*] Time range: 2014-08-01 ~ 2014-10-31
Full history
 [*] Segments: 90 co-avail
    * [400861-400948]: 90 (25.5% nan)
    * [400860-400861]: 90 (6.1% nan)
    * [400948-404050]: 90 (34.5% nan)
    * [404050-400950]: 90 (41.9% nan)
    * [401348-400860]: 90 (45.4% nan)
 [*] Examples (test): 18
 [*] Time range: 2014-08-01 ~ 2014-10-31
Gathered training: 3524
Gathered training: 918
